In [ ]:
# discriminative

class tsLSTM_discriminative():
    
    def __init__(self, n_lstm_dim, n_steps, n_data_dim, n_lstm_layers, session,\
                 lr, l2, max_norm ):
        
        self.LEARNING_RATE = lr
        self.L2 =  l2
        
        self.N_LSTM_LAYERS = n_lstm_layers
        self.N_LSTM_DIM    = n_lstm_dim
        
        self.N_STEPS    = n_steps
        self.N_DATA_DIM = n_data_dim
        
        self.x = tf.placeholder(tf.float32, [None, self.N_STEPS, self.N_DATA_DIM])
        self.y = tf.placeholder(tf.float32, [None, 1])
        
        self.keep_prob = tf.placeholder(tf.float32)
        
    
        
    
        with tf.variable_scope("lstm"):
            
#           !!change
            lstm_cell    = tf.contrib.rnn.GRUCell(self.N_LSTM_DIM, activation = tf.nn.relu)
#           state_is_tuple = True
            
            stacked_lstm = tf.contrib.rnn.MultiRNNCell([lstm_cell]* self.N_LSTM_LAYERS,\
                                                      state_is_tuple=True )
            
            self.hiddens, self.state = tf.nn.dynamic_rnn(cell = stacked_lstm,\
                                                         inputs = self.x,\
                                                         dtype = tf.float32)
            
        tmp_hiddens = tf.transpose( self.hiddens, [1,0,2]  )
        last_hidden = tmp_hiddens[-1]
        
        with tf.variable_scope("hidden"):
                
#           change  orthogonal ini
            w = tf.Variable(tf.random_normal([self.N_LSTM_DIM, 128],\
                            stddev=math.sqrt(2.0/self.N_LSTM_DIM)))
            b = tf.Variable(tf.zeros( [128] ))
            
            self.regularization = tf.nn.l2_loss(w)
            
            h = tf.matmul(last_hidden, w) + b
            h = tf.nn.relu( h )
        
        
        with tf.variable_scope("output"):
            
            w = tf.Variable(tf.random_normal([128, 1],\
                            stddev=math.sqrt(2.0/128)))
            b = tf.Variable(tf.zeros( [ 1 ] ))
            
            self.regularization += tf.nn.l2_loss(w)
            self.py = tf.matmul(h, w) + b
        
    
    def train_ini(self):
#       !!! change
        self.cost = tf.reduce_mean( tf.quare(self.y - self.py) ) + self.L2*self.regularization
        
        self.optimizer = \
        tf.train.AdamOptimizer(learning_rate = self.LEARNING_RATE).minimize(self.cost)  
#         tf.train.AdadeltaOptimizer(learning_rate = self.LEARNING_RATE).minimize(self.cost)
#         tf.train.AdamOptimizer(learning_rate = self.LEARNING_RATE).minimize(self.cost)  
#         !! same lr, converge faster
#         tf.train.AdadeltaOptimizer(learning_rate = self.LEARNING_RATE).minimize(self.cost)
#         tf.train.RMSPropOptimizer(learning_rate = self.LEARNING_RATE).minimize(self.cost)
#         tf.train.RMSPropOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.AdadeltaOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.AdamOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.GradientDescentOptimizer(learning_rate = self.lr).minimize(self.cost)
        
        self.init = tf.global_variables_initializer()
        sess.run( self.init )
        
        
    def train_batch(self, x_batch, y_batch, keep_prob ):
        
        _,c = sess.run([self.optimizer, self.cost],\
                        feed_dict={self.x:x_batch,\
                                   self.y:y_batch,\
                                   self.keep_prob:keep_prob\
                                 })
            
        return c

#   initialize inference         
    def inference_ini(self):

#       denormalzied RMSE  
        self.rmse = \
        tf.sqrt( tf.reduce_mean( tf.square( self.y - self.py ) ) )
        
#   infer givn testing data    
    def inference(self, x_test, y_test, keep_prob):
        return sess.run([self.rmse], feed_dict={self.x:x_test,\
                                                self.y:y_test,\
                                                self.keep_prob:keep_prob\
                                                })
        
   
    def test(self, x_test, y_test ):
        
        self.init = tf.global_variables_initializer()
        
        tmpshape = tf.shape(self.state)
        sess.run( self.init )
        
        return sess.run( [ tmpshape ], \
                 feed_dict={self.x:x_test, self.y:y_test})
    

In [ ]:
# discriminative

disc_xtrain = np.reshape( xtrain, [-1, 100, 1] )
disc_xtest  = np.reshape( xtest, [-1, 100, 1] )

disc_ytrain = ytrain
disc_ytest  = ytest

print np.shape(disc_xtrain), np.shape(disc_ytrain), np.shape(disc_xtest), \
np.shape(disc_ytest)


para_n_epoch = 1200

# tunable parameters

#   representation ability
para_lr = 0.001
para_lstm_dim = 256
para_lstm_layers = 1
#   regularization
para_batch_size = 32
para_l2 = 0.1

para_max_norm  = 4
para_keep_prob = 0.8

#  fixed parameters
para_data_dim = 1
para_steps = 100

# evaluation parameters
para_eval_byepoch = 10

tf.reset_default_graph()

with tf.Session() as sess:
    
    reg = tsLSTM_discriminative( para_lstm_dim, para_steps, para_data_dim, \
                                para_lstm_layers, sess, \
                                para_lr, para_l2, para_max_norm)
    
    reg.train_ini()
    reg.inference_ini()
    
    total_cnt   = np.shape(disc_xtrain)[0]
    total_batch = int(total_cnt/para_batch_size)
    
    total_idx = range(total_cnt)
    
#   training cycle
    for epoch in range(para_n_epoch):
        
        tmpc = 0.0
        
#       shuffle traning instances each epoch  
        np.random.shuffle(total_idx)
    
#       Loop over all batches
        for i in range(total_batch):
            
            batch_idx = total_idx[ i*para_batch_size: (i+1)*para_batch_size ] 
            
            batch_x = disc_xtrain[ batch_idx ]
            batch_y = disc_ytrain[ batch_idx ]            
            
            tmpc += reg.train_batch( batch_x, batch_y, para_keep_prob,)
        
        if epoch%para_eval_byepoch != 0:
            continue
    
        tmp_test_acc  = reg.inference( disc_xtest, disc_ytest,  para_keep_prob) 
        tmp_train_acc = reg.inference( disc_xtrain,disc_ytrain, para_keep_prob) 
        
        print "loss on epoch ", epoch, " : ", 1.0*tmpc/total_batch, tmp_test_acc,\
        tmp_train_acc
    
    print "Optimization Finished!"

In [ ]:
# generative

def expand_y( x, y ):
    cnt = len(x)
    expand_y = []
    
    for i in range(cnt):
        tmp = x[i][1:]
        tmp = np.append( tmp, y[i][0] )
        
        expand_y.append( tmp )
    
    return np.array( expand_y )

class tsLSTM_generative():
    
    def __init__(self, n_lstm_dim, n_steps, n_data_dim, n_lstm_layers, session,\
                 lr, l2, max_norm ):
        
        self.LEARNING_RATE = lr
        self.L2 =  l2
        
        self.N_LSTM_LAYERS = n_lstm_layers
        self.N_LSTM_DIM    = n_lstm_dim
        
        self.N_STEPS    = n_steps
        self.N_DATA_DIM = n_data_dim
        
        self.x = tf.placeholder(tf.float32, [None, self.N_STEPS, self.N_DATA_DIM])
        self.y = tf.placeholder(tf.float32, [None, self.N_STEPS])
        self.test_y    = tf.placeholder(tf.float32, [None, 1]
        self.keep_prob = tf.placeholder(tf.float32)
        
        with tf.variable_scope("lstm"):
            
            lstm_cell    = tf.contrib.rnn.GRUCell(self.N_LSTM_DIM)
#           , state_is_tuple = True
            
            stacked_lstm = tf.contrib.rnn.MultiRNNCell([lstm_cell]* self.N_LSTM_LAYERS,\
                                                      state_is_tuple=True )
            
            self.hiddens, self.state = tf.nn.dynamic_rnn(cell = stacked_lstm, \
                                                         inputs = self.x,\
                                                    dtype = tf.float32)
            
        with tf.variable_scope("output"):
            
            w = tf.Variable( tf.random_normal([self.N_LSTM_DIM, 1],\
                             stddev=math.sqrt(2.0/self.N_LSTM_DIM)) )
            
            b = tf.Variable(tf.zeros( [ 1 ] ))
            
            
            train_h  =  tf.reshape( self.hiddens, [ -1, self.N_LSTM_DIM ] )
            train_py =  tf.matmul( train_h, w ) + b
            self.py  =  tf.reshape( train_py, [-1, self.N_STEPS ] ) 
            
            
            test_h = tf.transpose( self.hiddens, [1,0,2] )
            self.test_py = tf.matmul( test_h[-1], w ) + b
        

    def train_ini(self):
        
        self.cost = tf.nn.l2_loss( self.y - self.py )
        
        self.optimizer = \
        tf.train.AdamOptimizer(learning_rate = self.LEARNING_RATE).minimize(self.cost)
#         tf.train.AdadeltaOptimizer(learning_rate = self.LEARNING_RATE).minimize(self.cost)
#         tf.train.RMSPropOptimizer(learning_rate = self.LEARNING_RATE).minimize(self.cost)
#         tf.train.AdamOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.RMSPropOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.AdadeltaOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.AdamOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.GradientDescentOptimizer(learning_rate = self.lr).minimize(self.cost)
        
        self.init = tf.global_variables_initializer()
        sess.run( self.init )
        
        
    def train_batch(self, x_batch, y_batch, keep_prob ):
        
        _,c = sess.run([self.optimizer, self.cost],\
                        feed_dict={self.x:x_batch,\
                                   self.y:y_batch,\
                                   self.keep_prob:keep_prob\
                                 })
            
        return c
    
#   initialize inference         
    def inference_ini(self):

#       denormalzied RMSE
        self.rmse = tf.sqrt( tf.reduce_mean(\
                             tf.square( self.test_y - self.test_py ) ) )
        
#   infer givn testing data    
    def inference(self, x_test, y_test, keep_prob):
        return sess.run([self.rmse], feed_dict={self.x:x_test,\
                                                self.test_y:y_test,\
                                                self.keep_prob:keep_prob\
                                                   })
        
   
    def test(self, x_test, y_test):
        
        self.init = tf.global_variables_initializer()
        
        tmpshape  = tf.shape(self.state)
        sess.run( self.init )
        
        return sess.run( [ tmpshape ], \
                         feed_dict={self.x:x_test, self.y:y_test})
    

In [ ]:
# generative

gen_ytrain_test =  ytrain
gen_ytrain = expand_y( xtrain, ytrain )
gen_ytest  = ytest

gen_xtrain = np.reshape( xtrain, [-1, 100, 1] )
gen_xtest  = np.reshape( xtest, [-1, 100, 1] )

print np.shape(gen_xtrain), np.shape(gen_ytrain), np.shape(gen_xtest), np.shape(gen_ytest)



para_n_epoch = 1200

# tunable parameters

#   representation ability
para_lr = 0.01
para_lstm_dim = 256
para_lstm_layers = 1
#   regularization
para_batch_size = 32
para_l2 = 0.1

para_max_norm = 4
para_keep_prob = 0.8

# fixed parameters
para_data_dim = 1
para_steps = 100

# evaluation parameters
para_eval_byepoch = 10

# clean the graph
tf.reset_default_graph()

with tf.Session() as sess:
    
    reg = tsLSTM_generative( para_lstm_dim, para_steps, para_data_dim, \
                                para_lstm_layers, sess, \
                                para_lr, para_l2, para_max_norm)
    
    reg.train_ini()
    reg.inference_ini()
    
    total_cnt   = np.shape(gen_xtrain)[0]
    total_batch = int(total_cnt/para_batch_size)
    
    total_idx = range(total_cnt)
    
 
    
#   training cycle
    for epoch in range(para_n_epoch):
        
        tmpc = 0.0
        
#       shuffle traning instances each epoch  
        np.random.shuffle(total_idx)
    
#       Loop over all batches
        for i in range(total_batch):
            
            batch_idx = total_idx[ i*para_batch_size: (i+1)*para_batch_size ] 
            
            batch_x = gen_xtrain[ batch_idx ]
            batch_y = gen_ytrain[ batch_idx ]            
            
            tmpc += reg.train_batch( batch_x, batch_y, para_keep_prob,)
        
        if epoch%para_eval_byepoch != 0:
            continue
    
        tmp_test_acc  = reg.inference( gen_xtest, gen_ytest,       para_keep_prob) 
        tmp_train_acc = reg.inference( gen_xtrain,gen_ytrain_test, para_keep_prob) 
        
        print "loss on epoch ", epoch, " : ", 1.0*tmpc/total_batch, tmp_test_acc,\
        tmp_train_acc
    
    print "Optimization Finished!"